In [18]:
!pip install pymupdf neo4j python-dateutil unidecode pandas

In [107]:
import os
from pathlib import Path
import re
import json
import uuid
from datetime import datetime
import textwrap

import fitz  # PyMuPDF
import pandas as pd
from dateutil import parser as dateparser
from unidecode import unidecode
from neo4j import GraphDatabase

# Will need to update below with New Neo4J Credentials

In [ ]:
# === CONFIGURATION =========================================================

# Root directory containing PDFs. Colab stores uploaded files under /content.
PDF_ROOT = "/content"   # will recursively search for all PDFs under here

# Where to write outputs
OUTPUT_DIR = "/content/conexus_outputs"
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

REVIEW_CSV_PATH   = os.path.join(OUTPUT_DIR, "case_studies_review.csv")
NDJSON_PATH       = os.path.join(OUTPUT_DIR, "case_studies.ndjson")
JSON_PATH         = os.path.join(OUTPUT_DIR, "case_studies.json")

print("PDF root:   ", PDF_ROOT)
print("Output dir: ", OUTPUT_DIR)
print("Review CSV: ", REVIEW_CSV_PATH)
print("NDJSON:     ", NDJSON_PATH)
print("JSON:       ", JSON_PATH)

# Neo4j Aura connection placeholders (fill these in when ready to load)
NEO4J_URI      = ""                                                         #<-- change
NEO4J_USER     = "neo4j"                                                    # or custom user
NEO4J_PASSWORD = ""                                                         # <-- change

PDF root:    /content
Output dir:  /content/conexus_outputs
Review CSV:  /content/conexus_outputs/case_studies_review.csv
NDJSON:      /content/conexus_outputs/case_studies.ndjson
JSON:        /content/conexus_outputs/case_studies.json


In [109]:
# === PARSING & TAGGING UTILITIES ==========================================

# Keyword-based tagging – refine these as you review real data
TECH_KEYWORDS = {
    "automation_general": ["automation", "automated", "automating"],
    "collaborative_robotics": ["cobot", "collaborative robot", "collaborative welding"],
    "industrial_robotics": ["robotic", "robot arm", "industrial robot"],
    "cnc_machining": [" cnc ", "cnc machine", "machining center"],
    "metal_additive": ["metal 3d", "metal additive", "metal printer"],
    "polymer_additive": ["3d printing", "additive manufacturing", "large-format printer"],
    "erp_mes": ["erp", "mes", "production system", "manufacturing execution"],
    "quality_control_software": ["quality software", "inspection software", "qc software"],
    "inventory_management_systems": ["inventory management", "real-time inventory", "barcode scanning"],
    "iot_sensors": ["iot", "sensor", "temperature monitoring", "data logger"],
    "machine_vision": ["machine vision", "vision system", "camera inspection"],
    "amr_agv": ["amr", "autonomous mobile robot", "agv"],
    "cold_chain_monitoring": ["cold chain", "temperature-controlled", "vaccine storage"],
}

OUTCOME_KEYWORDS = {
    "throughput_increase": ["increase throughput", "higher throughput", "more parts per hour"],
    "lead_time_reduction": ["reduced lead time", "shorter lead time", "cut lead times"],
    "scrap_reduction": ["reduce scrap", "less scrap", "lower scrap rate"],
    "capacity_expansion": ["increased capacity", "added capacity", "expand capacity"],
    "labor_efficiency": ["labor efficiency", "fewer operators", "same staff", "reassign workers"],
    "quality_improvement": ["improved quality", "fewer defects", "reduced rework"],
    "workforce_redeployment": ["redeploy", "reassign workers", "higher-value work"],
    "safety_improvement": ["safer", "reduced injuries", "improved safety"],
    "new_product_capability": ["new product", "new capability", "new offerings"],
    "new_markets": ["new market", "entered new", "export"],
    "data_visibility": ["real-time data", "dashboard", "visibility", "data-driven"],
    "supply_chain_resilience": ["reshoring", "onshoring", "supply chain resilien"],
}

PAGE_COUNTER_RX = re.compile(r"^\s*\d+\s*/\s*\d+\s*$")
DATE_HEADER_RX = re.compile(
    r"^(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s*\d{4}$"
)
URL_RX = re.compile(r"(https?://)?(www\.)?conexusindiana\.com/[^\s]+", re.I)

In [110]:
def normalize_line(s: str) -> str:
    s = unidecode(s).replace("\u00a0", " ").strip()
    return re.sub(r"\s+", " ", s)

In [111]:
def normalize_line(s: str) -> str:
    s = unidecode(s).replace("\u00a0", " ").strip()
    return re.sub(r"\s+", " ", s)

In [112]:
def extract_lines(pdf_path: str):
    """Return a list of cleaned lines, skipping obvious headers/footers."""
    doc = fitz.open(pdf_path)
    lines = []
    for p in doc:
        txt = p.get_text("text")
        for raw in txt.splitlines():
            line = normalize_line(raw)
            if not line:
                continue
            if PAGE_COUNTER_RX.match(line):
                continue
            if DATE_HEADER_RX.match(line):
                continue
            lines.append(line)
    doc.close()
    return lines

In [113]:
def extract_full_text(pdf_path: str) -> str:
    """Return full plain text from PDF."""
    doc = fitz.open(pdf_path)
    texts = []
    for page in doc:
        texts.append(page.get_text("text"))
    doc.close()
    return "\n".join(texts)

In [114]:
def detect_title(lines):
    """Heuristic: first non-URL, non-'Case Study' line near top."""
    for line in lines[:25]:
        low = line.lower()
        if URL_RX.search(line):
            continue
        if "case study" in low:
            continue
        if len(line) > 5 and any(c.isalpha() for c in line):
            return line
    return lines[0] if lines else None

In [115]:
def detect_conexus_url(lines):
    for line in lines:
        m = URL_RX.search(line)
        if m:
            return m.group(0)
    return None

In [116]:
def parse_award_date(text: str):
    """Parse an award date or quarter string into structured values."""
    text = text.strip()
    # Look for QX YYYY
    q = re.search(r"\bQ([1-4]),?\s*(\d{4})\b", text, re.I)
    if q:
        qn = int(q.group(1))
        yr = int(q.group(2))
        month = [3, 6, 9, 12][qn - 1]
        return {
            "award_quarter": f"Q{qn}",
            "award_year": yr,
            "award_date_iso": f"{yr:04d}-{month:02d}-01",
        }
    # Try full date parse
    try:
        dt = dateparser.parse(text, fuzzy=True)
        qn = (dt.month - 1) // 3 + 1
        return {
            "award_quarter": f"Q{qn}",
            "award_year": dt.year,
            "award_date_iso": dt.date().isoformat(),
        }
    except Exception:
        return {
            "award_quarter": None,
            "award_year": None,
            "award_date_iso": None,
        }

In [117]:
# List of valid Indiana counties (no "County" suffix)
VALID_IN_COUNTIES = [
    "Adams","Allen","Bartholomew","Benton","Blackford","Boone","Brown","Carroll","Cass",
    "Clark","Clay","Clinton","Crawford","Daviess","Dearborn","Decatur","DeKalb","Delaware",
    "Dubois","Elkhart","Fayette","Floyd","Fountain","Franklin","Fulton","Gibson","Grant",
    "Greene","Hamilton","Hancock","Harrison","Hendricks","Henry","Howard","Huntington",
    "Jackson","Jasper","Jay","Jefferson","Jennings","Johnson","Knox","Kosciusko","LaGrange",
    "Lake","LaPorte","Lawrence","Madison","Marion","Marshall","Martin","Miami","Monroe",
    "Montgomery","Morgan","Newton","Noble","Ohio","Orange","Owen","Parke","Perry","Pike",
    "Porter","Posey","Pulaski","Putnam","Randolph","Ripley","Rush","St. Joseph","Scott",
    "Shelby","Spencer","Starke","Steuben","Sullivan","Switzerland","Tippecanoe","Tipton",
    "Union","Vanderburgh","Vermillion","Vigo","Wabash","Warren","Warrick","Washington",
    "Wayne","Wells","White","Whitley",
]

# Build a canonical lookup (handles spacing differences like LaPorte / La Porte)
COUNTY_CANON = {}
for name in VALID_IN_COUNTIES:
    COUNTY_CANON[name.lower()] = name
    COUNTY_CANON[name.replace(" ", "").lower()] = name

In [118]:
def _normalize_county_candidate(raw_name: str):
    """
    Take a candidate like 'Hamilton County' or 'Indiana Furniture remains a home County'
    and return a clean 'Hamilton County' if it's valid, else None.
    """
    if not raw_name:
        return None

    s = raw_name.strip()
    # enforce 'County' suffix
    if not s.lower().endswith("county"):
        return None
    base = s[:-len("county")].strip()   # drop 'County'
    if not base:
        return None

    # 1) Try full base (e.g. 'St. Joseph', 'La Porte')
    key_full = base.lower()
    key_full_nospace = base.replace(" ", "").lower()

    if key_full in COUNTY_CANON:
        return COUNTY_CANON[key_full] + " County"
    if key_full_nospace in COUNTY_CANON:
        return COUNTY_CANON[key_full_nospace] + " County"

    # 2) Try last word only (e.g. 'Hamilton' from 'City of Hamilton')
    last = base.split()[-1]
    key_last = last.lower()
    key_last_nospace = last.replace(" ", "").lower()

    if key_last in COUNTY_CANON:
        return COUNTY_CANON[key_last] + " County"
    if key_last_nospace in COUNTY_CANON:
        return COUNTY_CANON[key_last_nospace] + " County"

    # Not a real Indiana county
    return None

In [119]:
def extract_county_from_lines(lines):
    """
    Try multiple patterns to find a county name from the text,
    then validate it against the Indiana county list.
    """
    norm_lines = [normalize_line(l) for l in lines]

    # Pass 1: "County: X" or "County - X"
    for line in norm_lines:
        if "county" in line.lower():
            m = re.search(r"County[^A-Za-z0-9]*([A-Z][A-Za-z\.' ]+)", line)
            if m:
                cand = m.group(1).strip()
                cand = re.sub(r"\s+", " ", cand)
                if not cand.lower().endswith("county"):
                    cand = cand + " County"
                name = _normalize_county_candidate(cand)
                if name:
                    return name

    # Pass 2: any 'X County' where X is one or more capitalized words
    for line in norm_lines:
        # e.g. "Hamilton County", "St. Joseph County"
        for m in re.finditer(r"\b([A-Z][a-z]+(?:\s+[A-Z][a-z\.]+)*)\s+County\b", line):
            cand = m.group(0).strip()
            cand = re.sub(r"\s+", " ", cand)
            name = _normalize_county_candidate(cand)
            if name:
                return name

    # Pass 3: label on its own line, value on previous/next line
    for i, line in enumerate(norm_lines):
        if line.lower() in {"county", "county:"}:
            for j in [i - 1, i + 1]:
                if 0 <= j < len(norm_lines):
                    neighbor = norm_lines[j].strip()
                    if not neighbor:
                        continue
                    # if neighbor already includes 'County'
                    if "county" in neighbor.lower():
                        m = re.search(r"\b([A-Z][a-z]+(?:\s+[A-Z][a-z\.]+)*)\s+County\b", neighbor)
                        if m:
                            cand = m.group(0).strip()
                            cand = re.sub(r"\s+", " ", cand)
                            name = _normalize_county_candidate(cand)
                            if name:
                                return name
                    else:
                        # treat neighbor as base name
                        cand = neighbor + " County"
                        cand = re.sub(r"\s+", " ", cand)
                        name = _normalize_county_candidate(cand)
                        if name:
                            return name

    # Pass 4: very last resort – scan whole text but still validate
    full_text = " \n".join(norm_lines)
    for m in re.finditer(r"\b([A-Z][a-z]+(?:\s+[A-Z][a-z\.]+)*)\s+County\b", full_text):
        cand = m.group(0).strip()
        cand = re.sub(r"\s+", " ", cand)
        name = _normalize_county_candidate(cand)
        if name:
            return name

    return None

In [120]:
def scan_key_stats(lines):
    """
    Heuristic extraction of Key Stats (grant amount, size, county, award info).
    Improved county detection using line-level patterns.
    """
    result = {
        "grant_amount_usd": None,
        "county_name": None,
        "company_size": None,
        "award_quarter": None,
        "award_year": None,
        "award_date_iso": None,
    }

    # Join all lines into a big string for some regexes
    text = " \n".join(lines)

    # --- Grant Amount ------------------------------------------------------
    m = re.search(r"Grant Amount[^$]*\$([\d,]+)", text, re.I)
    if m:
        try:
            amt = int(m.group(1).replace(",", ""))
            result["grant_amount_usd"] = amt
        except ValueError:
            pass

    # --- Company Size ------------------------------------------------------
    m = re.search(r"Company Size[^0-9]*([0-9]{1,5})", text, re.I)
    if m:
        try:
            result["company_size"] = int(m.group(1))
        except ValueError:
            pass

    # --- County (improved) -------------------------------------------------
    county = extract_county_from_lines(lines)
    result["county_name"] = county

    # --- Award date / quarter ----------------------------------------------
    m = re.search(r"Award Date[:\s]+([^\n]+)", text, re.I)
    award_info = None
    if m:
        award_info = parse_award_date(m.group(1))
    else:
        # Try to find something like "Q3 2022"
        m2 = re.search(r"\bQ[1-4],?\s*\d{4}\b", text)
        if m2:
            award_info = parse_award_date(m2.group(0))

    if award_info:
        result.update(award_info)

    return result

In [121]:
def detect_company_name_from_title(title: str):
    """Simple heuristic: company name is the leading part of the title before a dash."""
    if not title:
        return None
    parts = re.split(r" - | \u2013 | \u2014 ", title, maxsplit=1)
    company = parts[0].strip()
    return company or None

In [122]:
def extract_summary(lines, max_chars=500):
    """Use first few non-key-stat, non-url lines as a summary."""
    body = []
    for ln in lines:
        low = ln.lower()
        if "key stats" in low:
            continue
        if URL_RX.search(ln):
            continue
        if len(ln) < 4:
            continue
        body.append(ln)
        if len(" ".join(body)) >= max_chars:
            break
    text = " ".join(body)
    return (text[:max_chars] + "…") if len(text) > max_chars else text

In [123]:
def detect_tech_tags(full_text: str):
    text = full_text.lower()
    tags = set()
    for tag, kws in TECH_KEYWORDS.items():
        if any(kw.lower() in text for kw in kws):
            tags.add(tag)
    return sorted(tags)

In [124]:
def detect_outcome_tags(full_text: str):
    text = full_text.lower()
    tags = set()
    for tag, kws in OUTCOME_KEYWORDS.items():
        if any(kw.lower() in text for kw in kws):
            tags.add(tag)
    return sorted(tags)

In [125]:
def chunk_text(text: str, max_chars: int = 1200, overlap: int = 200):
    """
    Simple character-based chunker with overlap.
    - max_chars: target max length of each chunk
    - overlap:   how many characters to overlap between chunks
    """
    if not text:
        return []

    # Light whitespace normalization
    cleaned = re.sub(r"\s+", " ", text).strip()

    chunks = []
    start = 0
    n = len(cleaned)

    while start < n:
        end = min(start + max_chars, n)
        chunk_txt = cleaned[start:end].strip()
        if chunk_txt:
            chunks.append({
                "text": chunk_txt,
                "char_start": start,
                "char_end": end
            })
        if end == n:
            break
        # move forward but keep some overlap
        start = end - overlap

    return chunks

In [126]:
def build_case_record(pdf_path: str):
    """Build a structured record for a single case study PDF."""
    filename = os.path.basename(pdf_path)
    lines = extract_lines(pdf_path)
    full_text = extract_full_text(pdf_path)

    title = detect_title(lines)
    company_name = detect_company_name_from_title(title) if title else None
    url = detect_conexus_url(lines)
    key_stats = scan_key_stats(lines)
    summary = extract_summary(lines)

    case_id = str(uuid.uuid4())
    company_id = str(uuid.uuid4())

    # derive size_band from company_size
    size = key_stats["company_size"]
    if size is None:
        size_band = None
    elif size < 50:
        size_band = "small"
    elif size < 250:
        size_band = "medium"
    else:
        size_band = "large"

    tech_tags = detect_tech_tags(full_text)
    outcome_tags = detect_outcome_tags(full_text)

    # --- NEW: build chunks from full_text ---------------------------------
    raw_chunks = chunk_text(full_text, max_chars=1200, overlap=200)
    chunks = []
    for idx, ch in enumerate(raw_chunks):
        chunks.append({
            "chunk_id": str(uuid.uuid4()),
            "order": idx,
            "text": ch["text"],
            "char_start": ch["char_start"],
            "char_end": ch["char_end"],
        })
    # ----------------------------------------------------------------------

    record = {
        "case_id": case_id,
        "title": title,
        "company": {
            "company_id": company_id,
            "name": company_name,
            "size": size,
            "size_band": size_band,
            "industry_sector": None,
            "primary_products": None,
            "hq_city": None,
            "hq_county": key_stats["county_name"],
            "hq_state": "IN" if key_stats["county_name"] else None,
        },
        "grant": {
            "program_name": "Manufacturing Readiness Grants",
            "amount_usd": key_stats["grant_amount_usd"],
            "award_quarter": key_stats["award_quarter"],
            "award_year": key_stats["award_year"],
            "award_date_iso": key_stats["award_date_iso"],
        },
        "geo": {
            "county_name": key_stats["county_name"],
            "state": "IN" if key_stats["county_name"] else None,
        },
        "url": url,
        "pdf_filename": filename,
        "summary": summary,
        "full_text": full_text,
        "tech_tags": tech_tags,
        "outcome_tags": outcome_tags,
        "chunks": chunks,  # <-- NEW: list of chunk dicts
        "_extracted_at": datetime.utcnow().isoformat() + "Z",
    }

    return record


In [127]:
def find_all_pdfs(root: str):
    pdf_paths = []
    for dirpath, dirnames, filenames in os.walk(root):
        for fname in filenames:
            if fname.lower().endswith(".pdf"):
                pdf_paths.append(os.path.join(dirpath, fname))
    return sorted(pdf_paths)

In [128]:
def run_extraction(pdf_root: str, ndjson_out: str, json_out: str, review_csv: str):
    records = []
    pdf_files = find_all_pdfs(pdf_root)
    print(f"Found {len(pdf_files)} PDFs under {pdf_root}")

    with open(ndjson_out, "w", encoding="utf-8") as nd:
        for path in pdf_files:
            fname = os.path.basename(path)
            try:
                print(f"Parsing {path} ...")
                rec = build_case_record(path)
                records.append(rec)
                nd.write(json.dumps(rec, ensure_ascii=False) + "\n")
            except Exception as e:
                print(f"FAIL: {path} — {e}")

    # Write JSON array
    with open(json_out, "w", encoding="utf-8") as jf:
        json.dump(records, jf, ensure_ascii=False, indent=2)

    # Prepare CSV for review (flatten minimal fields + join tags)
    rows = []
    for r in records:
        rows.append({
            "case_id": r["case_id"],
            "pdf_filename": r["pdf_filename"],
            "title": r["title"],
            "company_name": r["company"]["name"],
            "company_size": r["company"]["size"],
            "size_band": r["company"]["size_band"],
            "county_name": r["geo"]["county_name"],
            "grant_amount_usd": r["grant"]["amount_usd"],
            "award_year": r["grant"]["award_year"],
            "award_quarter": r["grant"]["award_quarter"],
            "tech_tags": ", ".join(r["tech_tags"]),
            "outcome_tags": ", ".join(r["outcome_tags"]),
            "url": r["url"],
        })
    df = pd.DataFrame(rows)
    df.to_csv(review_csv, index=False)

    print(f"\nWrote {len(records)} records")
    print(f"NDJSON: {ndjson_out}")
    print(f"JSON:   {json_out}")
    print(f"CSV:    {review_csv}")
    return df

In [129]:
df_review = run_extraction(PDF_ROOT, NDJSON_PATH, JSON_PATH, REVIEW_CSV_PATH)

print("\nPreview of review CSV:")
df_review.head()

Found 80 PDFs under /content
Parsing /content/Arcamed, LLC - Conexus Indiana.pdf ...
Parsing /content/Decatur Plastic Products - Conexus Indiana.pdf ...
Parsing /content/Hightech Signs - Conexus Indiana.pdf ...
Parsing /content/Ikelite Underwater Systems - Conexus Indiana.pdf ...
Parsing /content/Jomar Machining & Fabricating - Conexus Indiana.pdf ...


/tmp/ipython-input-1373250659.py:75: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "_extracted_at": datetime.utcnow().isoformat() + "Z",


Parsing /content/Liberty Trailers - Conexus Indiana.pdf ...
Parsing /content/Michiana Global Mold - Conexus Indiana.pdf ...
Parsing /content/Mursix Corporation - Conexus Indiana.pdf ...
Parsing /content/POLARIS Laboratories® - Conexus Indiana.pdf ...
Parsing /content/Photon Automation - Conexus Indiana.pdf ...
Parsing /content/Tactile Engineering - Conexus Indiana.pdf ...
Parsing /content/Telamon Corporation - Conexus Indiana.pdf ...
Parsing /content/conexusindiana.com-AA Custom Automation.pdf ...
Parsing /content/conexusindiana.com-Accutech Mold  Machine.pdf ...
Parsing /content/conexusindiana.com-Addman Engineering.pdf ...
Parsing /content/conexusindiana.com-Advanced Cabinet Systems.pdf ...
Parsing /content/conexusindiana.com-Anu - Conexus Indiana.pdf ...
Parsing /content/conexusindiana.com-ArcticRx by ChefsFridge Co.pdf ...
Parsing /content/conexusindiana.com-Batesville Products Inc.pdf ...
Parsing /content/conexusindiana.com-California Closets Indianapolis Franchise.pdf ...
Parsing

,case_id,pdf_filename,title,company_name,company_size,size_band,county_name,grant_amount_usd,award_year,award_quarter,tech_tags,outcome_tags,url
0,832ca9d1-b3a6-496f-92e7-5a4ef18134e4,"Arcamed, LLC - Conexus Indiana.pdf","Arcamed, LLC - Conexus Indiana","Arcamed, LLC",4.0,small,Marion County,NaN,2025.0,Q4,"collaborative_robotics, erp_mes",,conexusindiana.com/case-study/arcamed-llc
1,1da14bfe-94a6-47ba-8170-4d06e7c5f067,Decatur Plastic Products - Conexus Indiana.pdf,Decatur Plastic Products,Decatur Plastic Products,2.0,small,Jennings County,NaN,NaN,None,"automation_general, erp_mes, industrial_robotics",,conexusindiana.com/case-study/decatur-plastic-...
2,4928faf0-41ac-4afc-ba38-0e9850305dc9,Hightech Signs - Conexus Indiana.pdf,Hightech Signs,Hightech Signs,4.0,small,Allen County,NaN,NaN,None,"automation_general, erp_mes",new_markets,conexusindiana.com/case-study/hightech-signs
3,6a269f31-af19-47a8-838c-25ee5f0c4b6c,Ikelite Underwater Systems - Conexus Indiana.pdf,Ikelite Underwater Systems,Ikelite Underwater Systems,3.0,small,Marion County,NaN,NaN,None,"automation_general, erp_mes",,conexusindiana.com/case-study/ikelite-underwat...
4,5d4428ff-03fa-48f0-bd75-779afe4c046c,Jomar Machining & Fabricating - Conexus Indian...,Jomar Machining & Fabricating,Jomar Machining & Fabricating,4.0,small,Elkhart County,NaN,NaN,None,"automation_general, erp_mes, industrial_robotics",,conexusindiana.com/case-study/jomar-machining-...


In [130]:
# === NEO4J CONSTRAINTS & CYPHER TEMPLATES =================================

CONSTRAINTS_CYPHER = """
CREATE CONSTRAINT company_id_unique IF NOT EXISTS
FOR (c:Company)
REQUIRE c.company_id IS UNIQUE;

CREATE CONSTRAINT case_id_unique IF NOT EXISTS
FOR (cs:CaseStudy)
REQUIRE cs.case_id IS UNIQUE;

CREATE CONSTRAINT tech_tag_name_unique IF NOT EXISTS
FOR (t:TagTech)
REQUIRE t.name IS UNIQUE;

CREATE CONSTRAINT outcome_tag_name_unique IF NOT EXISTS
FOR (o:TagOutcome)
REQUIRE o.name IS UNIQUE;

CREATE CONSTRAINT grant_program_name_unique IF NOT EXISTS
FOR (g:GrantProgram)
REQUIRE g.name IS UNIQUE;

CREATE CONSTRAINT county_name_unique IF NOT EXISTS
FOR (co:County)
REQUIRE co.name IS UNIQUE;

CREATE CONSTRAINT chunk_id_unique IF NOT EXISTS
FOR (ch:Chunk)
REQUIRE ch.chunk_id IS UNIQUE;
"""



UPSERT_CASE_CYPHER = """
MERGE (c:Company {company_id: $company_id})
SET c.name             = $company_name,
    c.size             = $company_size,
    c.size_band        = $company_size_band,
    c.industry_sector  = $company_industry_sector,
    c.primary_products = $company_primary_products,
    c.hq_city          = $company_hq_city,
    c.hq_state         = $company_hq_state

MERGE (co:County {name: $county_name})
ON CREATE SET co.state = $county_state

MERGE (c)-[:HEADQUARTERED_IN]->(co)

MERGE (cs:CaseStudy {case_id: $case_id})
SET cs.title          = $title,
    cs.summary        = $summary,
    cs.full_text      = $full_text,
    cs.pdf_filename   = $pdf_filename,
    cs.url            = $url,
    cs.grant_amount   = $grant_amount_usd,
    cs.award_quarter  = $award_quarter,
    cs.award_year     = $award_year,
    cs.award_date_iso = $award_date_iso,
    cs.extracted_at   = $extracted_at

MERGE (c)-[:HAS_CASE_STUDY]->(cs)

MERGE (g:GrantProgram {name: $program_name})
ON CREATE SET g.admin_org = "IEDC / Conexus Indiana"

MERGE (cs)-[:AWARDED_UNDER]->(g);
"""



TAGGING_CYPHER = """
MATCH (cs:CaseStudy {case_id: $case_id})

UNWIND $tech_tags AS techName
  MERGE (t:TagTech {name: techName})
  MERGE (cs)-[:USES_TECH]->(t)

WITH cs
UNWIND $outcome_tags AS outName
  MERGE (o:TagOutcome {name: outName})
  MERGE (cs)-[:HAS_OUTCOME]->(o);
"""

CHUNKS_CYPHER = """
MATCH (cs:CaseStudy {case_id: $case_id})
UNWIND $chunks AS ch
  MERGE (chunk:Chunk {chunk_id: ch.chunk_id})
  SET chunk.text       = ch.text,
      chunk.order      = ch.order,
      chunk.char_start = ch.char_start,
      chunk.char_end   = ch.char_end
  MERGE (cs)-[:HAS_CHUNK]->(chunk);
"""

In [131]:
def load_to_neo4j_from_json(json_path: str):
    # Read JSON records
    with open(json_path, "r", encoding="utf-8") as f:
        records = json.load(f)

    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

    with driver:
        # 1. Create constraints
        with driver.session() as session:
            print("Creating constraints (if not exist)...")
            for stmt in CONSTRAINTS_CYPHER.strip().split(";"):
                s = stmt.strip()
                if s:
                    session.run(s)
            print("Done.\n")

        # 2. Upsert case studies, tags, and chunks
        with driver.session() as session:
            for cs in records:
                comp   = cs["company"]
                grant  = cs["grant"]
                geo    = cs["geo"]
                chunks = cs.get("chunks", [])

                # Avoid null in County MERGE key
                county_name  = geo.get("county_name")
                if not county_name:
                    county_name = "Unknown County"
                county_state = geo.get("state") or "IN"

                params_case = {
                    "case_id": cs["case_id"],
                    "title": cs["title"],
                    "summary": cs["summary"],
                    "full_text": cs["full_text"],
                    "pdf_filename": cs["pdf_filename"],
                    "url": cs["url"],
                    "grant_amount_usd": grant["amount_usd"],
                    "award_quarter": grant["award_quarter"],
                    "award_year": grant["award_year"],
                    "award_date_iso": grant["award_date_iso"],
                    "extracted_at": cs["_extracted_at"],

                    "company_id": comp["company_id"],
                    "company_name": comp["name"],
                    "company_size": comp["size"],
                    "company_size_band": comp["size_band"],
                    "company_industry_sector": comp["industry_sector"],
                    "company_primary_products": comp["primary_products"],
                    "company_hq_city": comp["hq_city"],
                    "company_hq_state": comp["hq_state"],

                    "county_name": county_name,
                    "county_state": county_state,
                    "program_name": grant["program_name"],
                }

                # Upsert Company, County, CaseStudy, GrantProgram
                session.run(UPSERT_CASE_CYPHER, params_case)

                # Attach tags
                params_tags = {
                    "case_id": cs["case_id"],
                    "tech_tags": cs["tech_tags"],
                    "outcome_tags": cs["outcome_tags"],
                }
                session.run(TAGGING_CYPHER, params_tags)

                # Attach chunks (if present)
                if chunks:
                    params_chunks = {
                        "case_id": cs["case_id"],
                        "chunks": chunks,
                    }
                    session.run(CHUNKS_CYPHER, params_chunks)

                print(f"Loaded {cs['case_id']} ({cs['title']})")

    print("\nAll records (including chunks) loaded to Neo4j Aura.")


In [ ]:
# Uncomment this line AFTER you:
#  1) Upload PDFs under /content
#  2) Run the extraction cell
#  3) Verify the CSV/JSON look reasonable
#  4) Fill in NEO4J_URI / USER / PASSWORD

#load_to_neo4j_from_json(JSON_PATH)

Creating constraints (if not exist)...
Done.

Loaded 832ca9d1-b3a6-496f-92e7-5a4ef18134e4 (Arcamed, LLC - Conexus Indiana)
Loaded 1da14bfe-94a6-47ba-8170-4d06e7c5f067 (Decatur Plastic Products)
Loaded 4928faf0-41ac-4afc-ba38-0e9850305dc9 (Hightech Signs)
Loaded 6a269f31-af19-47a8-838c-25ee5f0c4b6c (Ikelite Underwater Systems)
Loaded 5d4428ff-03fa-48f0-bd75-779afe4c046c (Jomar Machining & Fabricating)
Loaded f11d06a7-3dce-4b04-b626-42117fe149e2 (Liberty Trailers)
Loaded 0d499362-5a98-4122-9b3d-fe32f1b6a16b (Michiana Global Mold - Conexus Indiana)
Loaded c01c0af8-ef90-4577-8ee7-6075952f215e (Mursix Corporation)
Loaded d200b157-ee6e-4382-ac8d-2d8531c8783a (POLARIS Laboratories(r))
Loaded 44968281-94fe-4290-a558-b8fe84f40499 (Photon Automation - Conexus Indiana)
Loaded 9e09d55b-a011-40a2-a781-1c52af74f6cd (Tactile Engineering - Conexus Indiana)
Loaded 80321284-3164-4279-adf1-e57f75c8383c (Telamon Corporation)
Loaded 8e8a824b-1221-4e84-be35-5361b48ece88 (A&A Custom Automation)
Loaded 16ea65